In [1]:
!pip install accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-fcykbu4l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-fcykbu4l
  Resolved https://github.com/huggingface/transformers to commit 25245ec26dc29bcf6102e1b4ddd0dfd02e720cf5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9140092 sha256=84a1ec06623fb2b5f99e22b609f9d799036581a6067f965da7bf8700e3814594
  Stored in directory: /tmp/pip-ephem-wheel-cache-2pk4k0g8/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
if torch.cuda.is_available():
    model_id = "meta-llama/Llama-2-7b-chat-hf"
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
    model.cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
def chat_with_llama(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to('cuda')
    output = model.generate(input_ids, max_length=256, num_beams=4, no_repeat_ngram_size=2)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [6]:
prompt = "teach me about lion in simple words"
response = chat_with_llama(prompt)


print("Llama:", response.split("\n")[1:])


Llama: ["Lions are large, carnivorous mammals that live in Africa and India. They are known for their distinctive manes, which are the thick hairs around the head, neck, and shoulders of males. The mane serves as a sign of masculinity and helps to protect the Lion's neck during fights.", '', 'Here are some interesting facts about Lions:', '1.Lion is the only cat species that lives in groups, called prides. A pride can consist of up to 30 lions, including several females, their cubs and one or more males', '2. Lionesses do most of the hunting, using coordinated attacks and stealth to bring down their prey. However, males also hunt and are responsible for protecting the pride and its territory', '3.The average lifespan of a Lion in the wild is around 12-16 years, although some have been known to live into their early 20s in good conditions. In captivity, they can live for upwards of25 years', '4.Despite their large size (males can weigh up 550 pounds), Liones']


In [7]:
raw_response = response.split("\n")[1:]


In [8]:
raw_response

["Lions are large, carnivorous mammals that live in Africa and India. They are known for their distinctive manes, which are the thick hairs around the head, neck, and shoulders of males. The mane serves as a sign of masculinity and helps to protect the Lion's neck during fights.",
 '',
 'Here are some interesting facts about Lions:',
 '1.Lion is the only cat species that lives in groups, called prides. A pride can consist of up to 30 lions, including several females, their cubs and one or more males',
 '2. Lionesses do most of the hunting, using coordinated attacks and stealth to bring down their prey. However, males also hunt and are responsible for protecting the pride and its territory',
 '3.The average lifespan of a Lion in the wild is around 12-16 years, although some have been known to live into their early 20s in good conditions. In captivity, they can live for upwards of25 years',
 '4.Despite their large size (males can weigh up 550 pounds), Liones']

In [10]:
def get_content(gr):
    content =""
    for i in gr:
        content+=i
    return content

In [11]:
get_content(raw_response)

"Lions are large, carnivorous mammals that live in Africa and India. They are known for their distinctive manes, which are the thick hairs around the head, neck, and shoulders of males. The mane serves as a sign of masculinity and helps to protect the Lion's neck during fights.Here are some interesting facts about Lions:1.Lion is the only cat species that lives in groups, called prides. A pride can consist of up to 30 lions, including several females, their cubs and one or more males2. Lionesses do most of the hunting, using coordinated attacks and stealth to bring down their prey. However, males also hunt and are responsible for protecting the pride and its territory3.The average lifespan of a Lion in the wild is around 12-16 years, although some have been known to live into their early 20s in good conditions. In captivity, they can live for upwards of25 years4.Despite their large size (males can weigh up 550 pounds), Liones"

In [12]:
content = get_content(raw_response)

In [13]:
content

"Lions are large, carnivorous mammals that live in Africa and India. They are known for their distinctive manes, which are the thick hairs around the head, neck, and shoulders of males. The mane serves as a sign of masculinity and helps to protect the Lion's neck during fights.Here are some interesting facts about Lions:1.Lion is the only cat species that lives in groups, called prides. A pride can consist of up to 30 lions, including several females, their cubs and one or more males2. Lionesses do most of the hunting, using coordinated attacks and stealth to bring down their prey. However, males also hunt and are responsible for protecting the pride and its territory3.The average lifespan of a Lion in the wild is around 12-16 years, although some have been known to live into their early 20s in good conditions. In captivity, they can live for upwards of25 years4.Despite their large size (males can weigh up 550 pounds), Liones"

In [14]:
!pip install gtts

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
from gtts import gTTS  
from playsound import playsound  
#rename the variable wrt the input from user
test_audio = gTTS(text=content, lang="en", slow=False,tld='com.au')
test_audio.save("lion_test.mp3")